<!DOCTYPE html>
<html>
<head>
<title>Image Grouping with Object Detection</title>
</head>
<body>
<h1>Image Grouping with Object Detection</h1>
<p>This HTML page provides an overview of the Python script that uses the Ultralytics YOLO library to detect objects in images and group them based on detected entities.</p>

<ol>
    <li>Import the necessary libraries: The script begins by importing the Ultralytics YOLO class (<code>ultralytics.YOLO</code>), the operating system module (<code>os</code>), the CSV module (<code>csv</code>), and the Image module from the Python Imaging Library (<code>PIL</code>).</li>
    <li>Load YOLO Model: The YOLO model is loaded using the pretrained model file named <code>'yolov8n.pt'</code>.</li>
    <li>Specify Folder Paths: The paths to the input folder (<code>folder_path</code>) where images are stored and the output folder (<code>output_folder</code>) where grouped images will be saved are defined.</li>
    <li>Retrieve File List: The <code>getFolder_Path(folder_path)</code> function is used to retrieve a list of file names from the specified input folder.</li>
    <li>Detect and Group Segments: The <code>detect_Segments()</code> function processes each image in the input folder, detects object segments using the YOLO model, counts entities, and groups images based on detected entities.</li>
    <li>Count Detected Entities: The <code>count_Entities(results)</code> function counts the detected entities and calculates the total number of detected objects.</li>
    <li>Group Images: The <code>group_images(img_name, counts)</code> function creates group folders based on entity IDs and copies images to the appropriate group folder.</li>
    <li>Main Module Execution: The script checks if it is being run as the main module, and if so, it calls the <code>detect_Segments()</code> function to initiate the detection and grouping process.</li>
</ol>

<p>This Python script uses the Ultralytics YOLO library to detect objects in images and group them based on detected entities. Grouped images are saved in separate folders corresponding to the detected entities.</p>
</body>
</html>


In [ ]:
from ultralytics import YOLO
import os
import csv
import shutil
from PIL import Image
 
# Load YOLO model
model = YOLO('yolov8n.pt')  # Using yolov8n pretrained model
folder_path = './All_Images'  # Path to folder where images are stored
output_folder = './Grouped_Images'  # Path to folder where grouped images will be saved
 
def getFolder_Path(folder_path):
    file_list = []
 
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            file_list.append(file_name)
    return file_list
 
def detect_Segments():
    files_in_folder = getFolder_Path(folder_path)
    for i in files_in_folder:
        path = f'./All_Images/{i}'
        results = model.predict(path, save=False, conf=0.4)
        counts, total_objects = count_Entities(results)
        group_images(i, counts)


def count_Entities(results):
    counts = {}
    total_objects = 0
 
    for result in results:
        boxes = result.boxes.cpu().numpy()
        total_objects += len(boxes)
        for box in boxes:
            cls = int(box.cls[0])
            if cls not in counts:
                counts[cls] = 1
            else:
                counts[cls] += 1
 
    return counts, total_objects
 

 
def group_images(img_name, counts):
    for entity_id in counts:
        entity_name = model.names[entity_id]
        group_folder = f"{output_folder}/Folder-grp{entity_id}"
        if not os.path.exists(group_folder):
            os.makedirs(group_folder)
 
        src_path = f"./All_Images/{img_name}"
        dst_path = f"{group_folder}/{img_name}"
        shutil.copy(src_path, dst_path)
 
    print(f"Images grouped for {img_name}")


if __name__ == "__main__":
    detect_Segments()